In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import os
#import matplotlib
#import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
from datetime import datetime
import pandas as pd
import random 
from torchvision.datasets import ImageFolder
import re
from torch.utils.data import Dataset, DataLoader

from PIL import Image
#from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import roc_auc_score
from skimage.io import imread, imsave
import skimage
from PIL import ImageFile
from PIL import Image
import io
import torchvision.transforms.functional as TF
from torch.autograd import Variable
import torch.nn.functional as F

def resize2d(img, size):
    return F.adaptive_avg_pool2d(Variable(img),size).data

torch.cuda.empty_cache()

In [2]:
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

########## Mean and std are calculated from the train dataset
normalize = transforms.Normalize(mean=[0.45271412, 0.45271412, 0.45271412],
                                     std=[0.33165374, 0.33165374, 0.33165374])
train_transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop((224),scale=(0.5,1.0)),
    transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(90),
    # random brightness and random contrast
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    normalize
])

val_transformer = transforms.Compose([
#     transforms.Resize(224),
#     transforms.CenterCrop(224),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

In [3]:
batchsize=4
def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data

class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):
        """
        Args:
            txt_path (string): Path to the txt file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        File structure:
        - root_dir
            - CT_COVID
                - img1.png
                - img2.png
                - ......
            - CT_NonCOVID
                - img1.png
                - img2.png
                - ......
        """
        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID','CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list    
        self.transform = transform
        len0=len(self.img_list)
        print('item,label=',self.img_list[len0-1][0],self.img_list[len0-1][1])
        print('item,label=',self.img_list[1][0],self.img_list[1][1])
        
    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        sample = {'img': image,
                  'label': int(self.img_list[idx][1])}
        return sample



    
if __name__ == '__main__':
    trainset = CovidCTDataset(root_dir='data',
                              txt_COVID='data/trainCT-COVID.txt',
                              txt_NonCOVID='data/trainCT-NonCOVID.txt',
                              transform= train_transformer)
    valset = CovidCTDataset(root_dir='data',
                              txt_COVID='data/valCT-COVID.txt',
                              txt_NonCOVID='data/valCT-NonCOVID.txt',
                              transform= val_transformer)
    testset = CovidCTDataset(root_dir='data',
                              txt_COVID='data/test_COVID.txt',
                              txt_NonCOVID='data/test_NonCOVID.txt',
                              transform= val_transformer)
    print(trainset.__len__())
    print(valset.__len__())
    print(testset.__len__())

    train_loader = DataLoader(trainset, batch_size=batchsize, drop_last=False, shuffle=True)
    val_loader = DataLoader(valset, batch_size=batchsize, drop_last=False, shuffle=False)
    test_loader = DataLoader(testset, batch_size=batchsize, drop_last=False, shuffle=False)
    

item,label= data/CT_NonCOVID/noncovid_ct_scan_0-53.jpg 1
item,label= data/CT_COVID/covid_ct_scan_0-116.jpg 0
item,label= data/CT_NonCOVID/noncovid_ct_scan_0-53.jpg 1
item,label= data/CT_COVID/covid_ct_scan_0-116.jpg 0
item,label= data/CT_COVID/Test_COVID/c78643e4-25.jpg 0
item,label= data/CT_COVID/Test_COVID/0eabf7e0-2.jpg 0
35
35
607


In [4]:
# Random Plots
train_no = trainset.__len__()
random_idx = np.random.randint(1, train_no, size=3)
#fig, axes = plt.subplots(3, 3, figsize=(16, 12))

#for idx, ax in enumerate(axes.ravel()):
for idx in range(3):
    #img = Image.open(train_list[idx])
    print('trainset=',trainset)
    sample = trainset.__getitem__(idx)
   # print('sample-shape=',sample)
    img = sample['img']
    label = sample['label']
    #img_path = trainset.__getitem__(idx).img_path
    print('img-path=,label=', idx, label)
    #ax.set_title(labels[idx])
   # ax.set_title(label)
   # ax.imshow(img)


#for batch_index, batch_samples in enumerate(train_loader):      
    #    data, target = batch_samples['img'], batch_samples['label']
#skimage.io.imshow(data[0,1,:,:].numpy())

trainset= <__main__.CovidCTDataset object at 0x7fef0be329a0>
img-path=,label= 0 0
trainset= <__main__.CovidCTDataset object at 0x7fef0be329a0>
img-path=,label= 1 0
trainset= <__main__.CovidCTDataset object at 0x7fef0be329a0>
img-path=,label= 2 0


In [5]:
#training process is defined here 

alpha = None
## alpha is None if mixup is not used
alpha_name = f'{alpha}'
device = 'cuda'

def train(optimizer, epoch):
    
    model.train()
    
    train_loss = 0
    train_correct = 0
    
    for batch_index, batch_samples in enumerate(train_loader):
        
        # move data to device
        data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
        
        ## adjust data to meet the input dimension of model
#         data = data[:, 0, :, :]
#         data = data[:, None, :, :]    
        
        #mixup
#         data, targets_a, targets_b, lam = mixup_data(data, target, alpha, use_cuda=True)
        
        
        optimizer.zero_grad()
        output = model(data)
        criteria = nn.CrossEntropyLoss()
        loss = criteria(output, target.long())
        
        #mixup loss
#         loss = mixup_criterion(criteria, output, targets_a, targets_b, lam)

        train_loss += criteria(output, target.long())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.long().view_as(pred)).sum().item()
    
        # Display progress and write to tensorboard
        if batch_index % bs == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()/ bs))
    
#     print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         train_loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
#         100.0 * train_correct / len(train_loader.dataset)))
#     f = open('model_result/{}.txt'.format(modelname), 'a+')
#     f.write('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         train_loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
#         100.0 * train_correct / len(train_loader.dataset)))
#     f.write('\n')
#     f.close()

In [6]:
#val process is defined here

def val(epoch):
    
    model.eval()
    test_loss = 0
    correct = 0
    results = []
    
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    
    
    criteria = nn.CrossEntropyLoss()
    # Don't update model
    with torch.no_grad():
        tpr_list = []
        fpr_list = []
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        # Predict
        for batch_index, batch_samples in enumerate(val_loader):
            data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
            
#             data = data[:, 0, :, :]
#             data = data[:, None, :, :]
            output = model(data)
            
            test_loss += criteria(output, target.long())
            score = F.softmax(output, dim=1)
            pred = output.argmax(dim=1, keepdim=True)
#             print('target',target.long()[:, 2].view_as(pred))
            correct += pred.eq(target.long().view_as(pred)).sum().item()
            
#             print(output[:,1].cpu().numpy())
#             print((output[:,1]+output[:,0]).cpu().numpy())
#             predcpu=(output[:,1].cpu().numpy())/((output[:,1]+output[:,0]).cpu().numpy())
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
           
    return targetlist, scorelist, predlist
    
    # Write to tensorboard
#     writer.add_scalar('Test Accuracy', 100.0 * correct / len(test_loader.dataset), epoch)

In [7]:
#test process is defined here 

def test(epoch):
    
    model.eval()
    test_loss = 0
    correct = 0
    results = []
    
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    
    
    criteria = nn.CrossEntropyLoss()
    # Don't update model
    with torch.no_grad():
        tpr_list = []
        fpr_list = []
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        # Predict
        for batch_index, batch_samples in enumerate(test_loader):
            data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
#             data = data[:, 0, :, :]
#             data = data[:, None, :, :]
#             print(target)
            output = model(data)
            
            test_loss += criteria(output, target.long())
            score = F.softmax(output, dim=1)
            pred = output.argmax(dim=1, keepdim=True)
#             print('target',target.long()[:, 2].view_as(pred))
            correct += pred.eq(target.long().view_as(pred)).sum().item()
#             TP += ((pred == 1) & (target.long()[:, 2].view_as(pred).data == 1)).cpu().sum()
#             TN += ((pred == 0) & (target.long()[:, 2].view_as(pred) == 0)).cpu().sum()
# #             # FN    predict 0 label 1
#             FN += ((pred == 0) & (target.long()[:, 2].view_as(pred) == 1)).cpu().sum()
# #             # FP    predict 1 label 0
#             FP += ((pred == 1) & (target.long()[:, 2].view_as(pred) == 0)).cpu().sum()
#             print(TP,TN,FN,FP)
            
            
#             print(output[:,1].cpu().numpy())
#             print((output[:,1]+output[:,0]).cpu().numpy())
#             predcpu=(output[:,1].cpu().numpy())/((output[:,1]+output[:,0]).cpu().numpy())
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
    return targetlist, scorelist, predlist
    
    # Write to tensorboard
#     writer.add_scalar('Test Accuracy', 100.0 * correct / len(test_loader.dataset), epoch)

In [8]:
"""Load Self-Trans model"""
"""Change names and locations to the Self-Trans.pt"""
import torchvision.models as models

model = models.densenet169(pretrained=True).cuda()
# pretrained_net = torch.load('model_backup/Dense169.pt')
# pretrained_net = torch.load('model_backup/mixup/Dense169_0.6.pt')
#pretrained_net = torch.load('model_backup/medical_transfer/medical_transfer_None_LUNA_moco_covid_moco.pt')
pretrained_net = torch.load('Self_Trans_xg.pt')


model.load_state_dict(pretrained_net)

modelname = 'Dense169_ssl_luna_moco'

In [9]:
# train
bs =batchsize
votenum = 1 #10
import warnings
warnings.filterwarnings('ignore')


r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(valset.__len__())
vote_score = np.zeros(valset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
                                             
scheduler = StepLR(optimizer, step_size=1)
#torch.save(model.state_dict(), "model_backup/medical_transfer/{}_{}_covid_moco_covid.pt".format(modelname,alpha_name)) 
        
total_epoch = 0; 2 #3000; # 0=no-training

for epoch in range(1, total_epoch+1):
    train(optimizer, epoch)
    
    targetlist, scorelist, predlist = val(epoch)
    print('target',targetlist)
    print('score',scorelist)
    print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        vote_score = vote_score/votenum
        
        print('vote_pred', vote_pred)
        print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUCp', roc_auc_score(targetlist, vote_pred))
        print('AUC', AUC)
        
        
        
#         if epoch == total_epoch:
        torch.save(model.state_dict(), "model_backup/medical_transfer/{}_{}_covid_moco_covid.pt".format(modelname,alpha_name))  
        torch.save(model.state_dict(), "Self_Trans_xg.pt")  
        
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},\
average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

#         f = open('model_result/medical_transfer/{}_{}.txt'.format(modelname,alpha_name), 'a+')
#         f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},\
# average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
#         epoch, r, p, F1, acc, AUC))
#         f.close()



In [10]:
# test

"""
bs = 10
import warnings
warnings.filterwarnings('ignore')

epoch = 1
r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())
targetlist, scorelist, predlist = test(epoch)
print('target',targetlist)
print('score',scorelist)
print('predict',predlist)
vote_pred = vote_pred + predlist 
vote_score = vote_score + scorelist 

TP = ((predlist == 1) & (targetlist == 1)).sum()

TN = ((predlist == 0) & (targetlist == 0)).sum()
FN = ((predlist == 0) & (targetlist == 1)).sum()
FP = ((predlist == 1) & (targetlist == 0)).sum()

print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
print('TP+FP',TP+FP)
p = TP / (TP + FP)
print('precision',p)
p = TP / (TP + FP)
r = TP / (TP + FN)
print('recall',r)
F1 = 2 * r * p / (r + p)
acc = (TP + TN) / (TP + TN + FP + FN)
print('F1',F1)
print('acc',acc)
AUC = roc_auc_score(targetlist, vote_score)
print('AUC', AUC)
"""

# f = open(f'model_result/medical_transfer/test_{modelname}_{alpha_name}_LUNA_moco_CT_moco.txt', 'a+')
# f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},\
# average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
# epoch, r, p, F1, acc, AUC))
# f.close()
# torch.save(model.state_dict(), "model_backup/medical_transfer/{}_{}_covid_moco_covid.pt".format(modelname,alpha_name))


"\nbs = 10\nimport warnings\nwarnings.filterwarnings('ignore')\n\nepoch = 1\nr_list = []\np_list = []\nacc_list = []\nAUC_list = []\n# TP = 0\n# TN = 0\n# FN = 0\n# FP = 0\nvote_pred = np.zeros(testset.__len__())\nvote_score = np.zeros(testset.__len__())\ntargetlist, scorelist, predlist = test(epoch)\nprint('target',targetlist)\nprint('score',scorelist)\nprint('predict',predlist)\nvote_pred = vote_pred + predlist \nvote_score = vote_score + scorelist \n\nTP = ((predlist == 1) & (targetlist == 1)).sum()\n\nTN = ((predlist == 0) & (targetlist == 0)).sum()\nFN = ((predlist == 0) & (targetlist == 1)).sum()\nFP = ((predlist == 1) & (targetlist == 0)).sum()\n\nprint('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)\nprint('TP+FP',TP+FP)\np = TP / (TP + FP)\nprint('precision',p)\np = TP / (TP + FP)\nr = TP / (TP + FN)\nprint('recall',r)\nF1 = 2 * r * p / (r + p)\nacc = (TP + TN) / (TP + TN + FP + FN)\nprint('F1',F1)\nprint('acc',acc)\nAUC = roc_auc_score(targetlist, vote_score)\nprint('AUC', AUC)\n"

In [11]:
## saving testing results


## testing on test-dataset
test_no = testset.__len__()
print('test_no=',test_no)
#random_idx = np.random.randint(1, train_no, size=3)
#fig, axes = plt.subplots(3, 3, figsize=(16, 12))
with io.open(f'test_DenseNet_Jun30-2.txt', 'w') as f:
#for idx, ax in enumerate(axes.ravel()):

    for item in read_txt('data/test_COVID.txt'):
        #print('item=',item)
        img_path = os.path.join('data',item)
        img = Image.open(img_path).convert('RGB')

    #for idx in range (10):    #range(test_no):
        #img = Image.open(train_list[idx])
       # sample = trainset.__getitem__(idx)
       # print('sample-shape=',sample)
       # img = sample['img']
        img1 = TF.to_tensor(img)
        img2 = resize2d(img1,(224,224))
        x = torch.zeros(1,3,224,224)
        x[0] = img2
        out = x[0]
       # out = img
        out.unsqueeze_(0)
        out = out.to('cuda:0')
        model.to('cuda:0')
        label = sample['label']
        #img_path = trainset.__getitem__(idx).img_path  
        output = model(out)
        cls = torch.max(output[0],0)
        cls = cls.indices
        #print('label, output=', label, cls)
        if cls>0:
            f.write("%s\t%s\n" % (item, '0'))
        else:
            f.write("%s\t%s\n" % (item, '1'))
        #ax.set_title(labels[idx])
       # ax.set_title(label)
       # ax.imshow(img)



"""
train_no = trainset.__len__()
print('train_no=',train_no)
#random_idx = np.random.randint(1, train_no, size=3)
#fig, axes = plt.subplots(3, 3, figsize=(16, 12))
#for idx, ax in enumerate(axes.ravel()):

with io.open('train_results_Jun23.txt', 'w') as f:
    for item in read_txt('data/trainCT-COVID.txt'):
        #print('item=',item)
        img_path = os.path.join('data/CT_COVID',item)
        img = Image.open(img_path).convert('RGB')

    #for idx in range (10):    #range(test_no):
        #img = Image.open(train_list[idx])
       # sample = trainset.__getitem__(idx)
       # print('sample-shape=',sample)
       # img = sample['img']
        img1 = TF.to_tensor(img)
        img2 = resize2d(img1,(224,224))
        x = torch.zeros(1,3,224,224)
        x[0] = img2
        out = x[0]
       # out = img
        out.unsqueeze_(0)
        out = out.to('cuda:0')
        model.to('cuda:0')
        label = sample['label']
        #img_path = trainset.__getitem__(idx).img_path  
        output = model(out)
        cls = torch.max(output[0],0)
        cls = cls.indices
        #print('label, output=', label, cls)
        if cls>0:
            f.write("%s\t%s\n" % (item, '1'))
        else:
            f.write("%s\t%s\n" % (item, '0'))
    #ax.set_title(labels[idx])
   # ax.set_title(label)
   # ax.imshow(img)
"""


#for batch_index, batch_samples in enumerate(train_loader):      
#        data, target = batch_samples['img'], batch_samples['label']
#skimage.io.imshow(data[0,1,:,:].numpy())
# f = open(f'model_result/medical_transfer/test_{modelname}_{alpha_name}_LUNA_moco_CT_moco.txt', 'a+')
# f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},\
# average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
# epoch, r, p, F1, acc, AUC))
# f.close()
# torch.save(model.state_dict(), "model_backup/medical_transfer/{}_{}_covid_moco_covid.pt".format(modelname,alpha_name))


test_no= 607


'\ntrain_no = trainset.__len__()\nprint(\'train_no=\',train_no)\n#random_idx = np.random.randint(1, train_no, size=3)\n#fig, axes = plt.subplots(3, 3, figsize=(16, 12))\n#for idx, ax in enumerate(axes.ravel()):\n\nwith io.open(\'train_results_Jun23.txt\', \'w\') as f:\n    for item in read_txt(\'data/trainCT-COVID.txt\'):\n        #print(\'item=\',item)\n        img_path = os.path.join(\'data/CT_COVID\',item)\n        img = Image.open(img_path).convert(\'RGB\')\n\n    #for idx in range (10):    #range(test_no):\n        #img = Image.open(train_list[idx])\n       # sample = trainset.__getitem__(idx)\n       # print(\'sample-shape=\',sample)\n       # img = sample[\'img\']\n        img1 = TF.to_tensor(img)\n        img2 = resize2d(img1,(224,224))\n        x = torch.zeros(1,3,224,224)\n        x[0] = img2\n        out = x[0]\n       # out = img\n        out.unsqueeze_(0)\n        out = out.to(\'cuda:0\')\n        model.to(\'cuda:0\')\n        label = sample[\'label\']\n        #img_path =